In [1]:
pip install -U sentence-transformers

In [5]:
from sentence_transformers import SentenceTransformer

In [7]:
# Load a pretrained Sentence Transformer model
model = SentenceTransformer('all-MiniLM-l6-v2')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [137]:
# The sentence and labels to encode
sentence = [ "Hey mom, im really hungry. Whats for dinner tonight?"]
labels = ['sports', 'halloween','video games', 'juice','cheese burger', 'money', 'lonely', 'afraid', 'happy', 'angry', 'space', 'pizza', 'steak', 'sushi']
# encode sentences
sentence_embedding = model.encode(sentence)
# encode labels
label_embedding = model.encode(labels)

In [138]:
# Print embedding shapes

print(f"sentence embedding shape: {sentence_embedding.shape}")
print(f"Label embedding shape: {label_embedding.shape}")

sentence embedding shape: (1, 384)
Label embedding shape: (14, 384)


In [139]:
# Get embedding similarity between sentences and labels
similarities = model.similarity(sentence_embedding, label_embedding)
print(similarities)

tensor([[0.1273, 0.2058, 0.1156, 0.2555, 0.3880, 0.1729, 0.1675, 0.1336, 0.1696,
         0.1814, 0.0946, 0.4179, 0.3522, 0.2498]])


In [140]:
print(similarities.shape)

torch.Size([1, 14])


In [141]:
import torch

In [142]:
# Raw similarity scores
similarities = similarities.squeeze()
similarities = torch.relu(similarities)
label_similarity_dict = {}
for similarity, label in zip(similarities, labels):
  label_similarity_dict[label] = round( similarity.item(),3)

In [143]:
label_similarity_dict

{'sports': 0.127,
 'halloween': 0.206,
 'video games': 0.116,
 'juice': 0.256,
 'cheese burger': 0.388,
 'money': 0.173,
 'lonely': 0.167,
 'afraid': 0.134,
 'happy': 0.17,
 'angry': 0.181,
 'space': 0.095,
 'pizza': 0.418,
 'steak': 0.352,
 'sushi': 0.25}

In [144]:
import numpy as np
from numpy.typing import NDArray

In [145]:
def top_n_highest_similarities(n:int,
                              input_array:NDArray[np.float32]) -> NDArray[np.int_]:
  """ Takes similarity score and returns the top k similar labels

  Args:
    int: n represents how many scores you want returned
    input_array: array of similarity scores.

  Returns:
    array of top n similarity scores.
  """
  return np.argsort(input_array)[-n:][::-1]




In [146]:
arr_similarities = similarities.numpy()

In [147]:
type(arr_similarities)
arr_similarities

array([0.1272824 , 0.20578067, 0.11561131, 0.2555341 , 0.3880444 ,
       0.17290929, 0.16747968, 0.13363779, 0.16955262, 0.18137226,
       0.09460753, 0.41787398, 0.35215974, 0.24984065], dtype=float32)

In [148]:
# Normalize similarity probabilites to get score between 0 and 1
arr_similarities = similarities.numpy()
top_n_predicted_indices = top_n_highest_similarities(n = 5, input_array = similarities.numpy() )
top_n_predicted_indices


array([11,  4, 12,  3, 13])

In [159]:
top_n_similarities = [ (index, arr_similarities[index], labels[index]) for index in top_n_predicted_indices]
top_n_similarities

[(np.int64(11), np.float32(0.41787398), 'pizza'),
 (np.int64(4), np.float32(0.3880444), 'cheese burger'),
 (np.int64(12), np.float32(0.35215974), 'steak'),
 (np.int64(3), np.float32(0.2555341), 'juice'),
 (np.int64(13), np.float32(0.24984065), 'sushi')]

In [160]:
top_n_similarities

[(np.int64(11), np.float32(0.41787398), 'pizza'),
 (np.int64(4), np.float32(0.3880444), 'cheese burger'),
 (np.int64(12), np.float32(0.35215974), 'steak'),
 (np.int64(3), np.float32(0.2555341), 'juice'),
 (np.int64(13), np.float32(0.24984065), 'sushi')]

In [161]:
top_n_labels = [element[2] for element in top_n_similarities]
top_n_similarity_scores = [element[1] for element in top_n_similarities]
top_n_similarity_scores

[np.float32(0.41787398),
 np.float32(0.3880444),
 np.float32(0.35215974),
 np.float32(0.2555341),
 np.float32(0.24984065)]

In [162]:

print(f"Sentence is: {sentence}")
print(f"top n predicted labels are : {top_n_labels}")
print(f"top n similarity scores are : {top_n_similarity_scores}")

Sentence is: ['Hey mom, im really hungry. Whats for dinner tonight?']
top n predicted labels are : ['pizza', 'cheese burger', 'steak', 'juice', 'sushi']
top n similarity scores are : [np.float32(0.41787398), np.float32(0.3880444), np.float32(0.35215974), np.float32(0.2555341), np.float32(0.24984065)]
